In [1]:
from lxml import html
import lxml
from lxml.etree import tostring
from bs4 import BeautifulSoup
import time
import requests
import pickle
import re
import pandas as pd

In [2]:
liens = pickle.load(open('liens_filtres.pickle', 'rb'))

In [3]:
columns = ['lien', 'doc_type', 'date', 'content']
df_output = pd.DataFrame(columns=columns)

In [4]:
def convert_to_text(content):
    raw_html = tostring(content)
    soup = BeautifulSoup(raw_html, "lxml")
    text = soup.get_text()
    text = re.sub('\s+', ' ', text)
    return text

In [5]:
def find_type(lien):
    matches = re.findall(r"(?:[A-Z])+(?=[:-])", lien)
    if (len(matches) > 0):
        return matches[0]
    return ''

In [6]:
def get_date(date):
    if (len(date) > 0):
        return date[0]
    return ''

In [7]:
def get_row(lien):
    url = 'http://europa.eu'+lien
    response = requests.get(url)
    if(response.status_code == 200):
        content_source = response.text.encode("utf-8")
        xml_doc = html.document_fromstring(content_source)

        doc_type = find_type(lien)

        date = xml_doc.xpath("//p[@class = 'date']/text()")
        date = get_date(date)

        content_tree = xml_doc.xpath("//div[@id = 'contentPressRelease']")
        content = convert_to_text(content_tree[0])

        row = [lien, doc_type, date, content]
        return row
    return False

In [8]:
index = 1
for lien in liens:
    print(lien)
    doc = get_row(lien)
    if (doc):
        df_output.loc[index] = get_row(lien)
        index = index + 1
        time.sleep(2)

/rapid/press-release_AGENDA-19-845_en.htm
/rapid/press-release_IP-19-850_en.htm
/rapid/press-release_MEX-19-902_en.htm
/rapid/press-release_SPEECH-19-905_en.htm
/rapid/press-release_MEMO-18-6463_en.htm
/rapid/press-release_MEX-18-6506_en.htm
/rapid/press-release_SPEECH-18-6509_en.htm
/rapid/press-release_SPEECH-18-6508_en.htm
/rapid/press-release_IP-18-6254_en.htm
/rapid/press-release_MEX-18-6343_en.htm
/rapid/press-release_MEMO-18-6175_en.htm
/rapid/press-release_MEX-18-3926_en.htm
/rapid/press-release_MEMO-18-1581_en.htm
/rapid/press-release_IP-18-604_en.htm
/rapid/press-release_SPEECH-18-686_en.htm
/rapid/press-release_MEX-17-5369_en.htm
/rapid/press-release_MEMO-17-4682_en.htm
/rapid/press-release_MEX-17-4847_en.htm
/rapid/press-release_SPEECH-17-4780_en.htm
/rapid/press-release_SPEECH-17-4779_en.htm
/rapid/press-release_AGENDA-17-4361_en.htm
/rapid/press-release_IP-17-4362_en.htm
/rapid/press-release_MEX-17-4503_en.htm
/rapid/press-release_SPEECH-17-4482_en.htm
/rapid/press-releas

In [9]:
df_output.head()

,lien,doc_type,date,content
1,/rapid/press-release_AGENDA-19-845_en.htm,AGENDA,,European Commission - Upcoming events Thursda...
2,/rapid/press-release_IP-19-850_en.htm,IP,"Brussels, 7 February 2019",European Commission - Press release Winter 20...
3,/rapid/press-release_MEX-19-902_en.htm,MEX,"Brussels, 7 February 2019",European Commission - Daily News Daily News 0...
4,/rapid/press-release_SPEECH-19-905_en.htm,SPEECH,"Brussels, 7 February 2019",European Commission - Speech - [Check Against...
5,/rapid/press-release_MEMO-18-6463_en.htm,MEMO,"Brussels, 21 November 2018",European Commission - Fact Sheet European Sem...


In [11]:
# df_output.to_csv('ecb.csv', index=False, header=True)

In [12]:
# Un lien n'est pas valide -> 116 pages au lieu de 117
df_output.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116 entries, 1 to 116
Data columns (total 4 columns):
lien        116 non-null object
doc_type    116 non-null object
date        116 non-null object
content     116 non-null object
dtypes: object(4)
memory usage: 4.5+ KB


In [10]:
with open('df_articles.pickle', 'wb') as handle:
    pickle.dump(df_output, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# tests
response = requests.get('http://europa.eu/rapid/press-release_AGENDA-19-845_en.htm')
content_source = response.text.encode("utf-8")
xml_doc = html.document_fromstring(content_source)
content = xml_doc.xpath("//div[@id = 'contentPressRelease']")

In [ ]:
response.status_code